In [2]:
import numpy as np

In [ ]:
from scipy.spatial import Delaunay
import time

for d in [5, 6, 7, 8, 9, 10]:
    for i in [100, 200, 500]:
        dataset = np.random.rand(i, d)
        t = time.time()
        tri = Delaunay(dataset)
        t = time.time() - t
        print(i, d, t)

100 5 0.05619049072265625
200 5 0.18735527992248535
500 5 0.795891284942627
100 6 0.29929089546203613
200 6 1.193056344985962
500 6 5.092738389968872
100 7 1.4879753589630127
200 7 7.54082989692688
500 7 48.17451882362366
100 8 7.500083923339844
200 8 39.789979219436646
500 8 282.7473111152649
100 9 31.47176766395569
200 9 246.07852029800415


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.spatial.distance import  pdist, squareform
import bisect

def get_gong(df, y=0.5):
    # df is a pandas dataframe who has x and y as columns
    # y is the gamma parameter 
    graph = nx.DiGraph() 
    graph.add_nodes_from(df.iterrows())
    dists = pdist(df[['x','y']]) 
        dists = squareform(dists)
    y_dists = (1-y)*dists

    for node_a, row_a in df.iterrows():
        node_a_coord = list(row_a[:2]) # O(dn)
        dist_idx = np.argsort(dists[node_a]) # O(nlog n)
        for node_b in dist_idx:
            if node_a == node_b:
                continue

            node_b_coord = list(df.loc[node_b, ['x','y']])

            d_i = y_dists[node_a][node_b]
            first_greater = bisect.bisect_left(dists[node_a][dist_idx], d_i)

            b_is_GONG = True

            for node_j in dist_idx[:first_greater]:
                if node_a == node_j:
                    continue

                d_j = y_dists[node_a][node_j]

                if d_j < d_i:
                    b_is_GONG = False
                    break # node_j could be a GONG

            if b_is_GONG:
                graph.add_edge(node_a, node_b, weight = dists[node_a][node_b])
    return graph